In [ ]:
import numpy as np
import random
import math

# Number of cities
NUM_CITIES = 50

# Generate random city coordinates
random.seed(0)  # For reproducibility
cities = np.random.rand(NUM_CITIES, 2)  # Random coordinates in [0, 1]


In [ ]:
def distance(city1, city2):
    return math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# Compute distances between all cities
distances = np.zeros((NUM_CITIES, NUM_CITIES))
for i in range(NUM_CITIES):
    for j in range(NUM_CITIES):
        distances[i][j] = distance(cities[i], cities[j])

# Initialize pheromone levels on edges
pheromones = np.ones((NUM_CITIES, NUM_CITIES))

In [ ]:
# ACO Parameters
NUM_ANTS = 20
MAX_ITER = 100
RHO = 0.1    # Evaporation rate
Q = 100      # Pheromone deposit factor

# Function to perform Ant Colony Optimization
def ant_colony_optimization(distances, pheromones, alpha, beta):
    num_cities = len(distances)
    best_tour = None
    best_length = float('inf')

    for _ in range(MAX_ITER):
        ant_tours = []
        tour_lengths = []

        # Construct solutions for each ant
        for ant in range(NUM_ANTS):
            visited = np.zeros(num_cities, dtype=bool)
            tour = []
            current_city = random.randint(0, num_cities - 1)
            tour.append(current_city)
            visited[current_city] = True

            # Build the tour for the current ant
            for _ in range(num_cities - 1):
                probabilities = np.zeros(num_cities)

                # Calculate probabilities for the next city
                for next_city in range(num_cities):
                    if not visited[next_city]:
                        pheromone = pheromones[current_city][next_city]
                        heuristic = 1.0 / distances[current_city][next_city]
                        probabilities[next_city] = (pheromone ** alpha) * (heuristic ** beta)

                probabilities = probabilities / probabilities.sum()
                next_city = np.random.choice(range(num_cities), p=probabilities)
                tour.append(next_city)
                visited[next_city] = True
                current_city = next_city

            ant_tours.append(tour)
            tour_length = sum(distances[tour[i]][tour[i + 1]] for i in range(num_cities - 1)) + distances[tour[-1]][tour[0]]
            tour_lengths.append(tour_length)

            # Update pheromone levels
            for i in range(num_cities):
                pheromones[tour[i]][tour[(i + 1) % num_cities]] += Q / tour_length

            # Update best tour
            if tour_length < best_length:
                best_tour = tour
                best_length = tour_length

        # Evaporate pheromone
        pheromones *= (1 - RHO)

    return best_length


In [ ]:
# PSO Parameters
NUM_PARTICLES = 10
MAX_ITER_PSO = 10
W = 0.7
C1 = 2.0
C2 = 2.0

# Function to sample a subset of the cities
def sample_subset(distances, subset_size=10):
    indices = np.random.choice(range(len(distances)), subset_size, replace=False)
    subset_distances = distances[np.ix_(indices, indices)]
    return subset_distances

# Objective function for PSO (Minimize tour length on subset)
def objective_function(alpha, beta):
    subset_distances = sample_subset(distances, subset_size=10)
    subset_pheromones = np.ones(subset_distances.shape)
    return ant_colony_optimization(subset_distances, subset_pheromones, alpha, beta)

# PSO initialization
particles = np.random.rand(NUM_PARTICLES, 2)  # Random initialization of particles
velocities = np.zeros((NUM_PARTICLES, 2))
pbest_positions = particles.copy()
pbest_values = np.full(NUM_PARTICLES, float('inf'))
gbest_position = np.zeros(2)
gbest_value = float('inf')

# PSO main loop
for _ in range(MAX_ITER_PSO):
    for i in range(NUM_PARTICLES):
        alpha, beta = particles[i]
        fitness = objective_function(alpha, beta)

        # Update personal best
        if fitness < pbest_values[i]:
            pbest_values[i] = fitness
            pbest_positions[i] = particles[i]

        # Update global best
        if fitness < gbest_value:
            gbest_value = fitness
            gbest_position = particles[i]

        # Update velocities and positions
        velocities[i] = (W * velocities[i] +
                         C1 * random.random() * (pbest_positions[i] - particles[i]) +
                         C2 * random.random() * (gbest_position - particles[i]))
        particles[i] += velocities[i]

# Extract best parameters from PSO
best_alpha, best_beta = gbest_position

# Validate best parameters on the full dataset
best_length = ant_colony_optimization(distances, pheromones, best_alpha, best_beta)
print(f"Best parameters (alpha, beta): ({best_alpha}, {best_beta})")
print(f"Length of best tour found: {best_length}")


Best parameters (alpha, beta): (0.9688413428396695, 1.778582253879069)
Length of best tour found: 5.490440191467256
